In [ ]:
%pwd

In [1]:
import os
os.chdir("../")
%pwd

'd:\\Projects\\Deep_learning\\Deep-learning-model'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: Path
    params_imag_size: list

#n
@dataclass(frozen= True)
class PrepareCallbacksConfig:
    root_dir:Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath:Path

In [3]:
from Hand_gesture_clf.utils.common import read_yaml, create_diorectories
from Hand_gesture_clf.constants import *
import tensorflow as tf


In [15]:
class CnfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_diorectories([self.config.artifacts_root])


    def get_prepare_callbacksd_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_chkpt_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_diorectories([
            Path(model_chkpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir= Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(config.checkpoint_model_filepath)
        )
        return prepare_callback_config
    

    def get_training_config(self)->TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "HandGesture\images")

        create_diorectories([Path(training.root_dir)])


        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTATION,
            params_imag_size = params.IMAGE_SIZE
         )
        
        return training_config


        

In [16]:
import time
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp =time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbaks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath= str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )
    
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbaks
        ]

In [20]:
class Training:
    def __init__(self, config:TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):
        data_generator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.25
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_imag_size[:-1],
            batch_size =self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip= True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **data_generator_kwargs
            )
        else:
            train_data_generator = valid_datagenerator

        self.train_generator = train_data_generator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model:tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = 20
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch  = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator,
            callbacks = callback_list

        )

        self.save_model(
            path = self.config.trained_model_path,
            model = self.model
        )



In [21]:
try:
    config = CnfigurationManager()
    prepare_callback_config = config.get_prepare_callbacksd_config()
    prepare_callback = PrepareCallbacks(config = prepare_callback_config)
    call_back_list = prepare_callback.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=call_back_list
    )

except Exception as e:
    raise e

[2024-10-03 02:02:19,125 : INFO: common: yaml file config\config.yaml loaded successfully]
[2024-10-03 02:02:19,136 : INFO: common: yaml file params.yaml loaded successfully]
[2024-10-03 02:02:19,140 : INFO: common: Created directory at : artifacts]
[2024-10-03 02:02:19,143 : INFO: common: Created directory at : artifacts\prepare_callbacks\checkpoint_dir]
[2024-10-03 02:02:19,147 : INFO: common: Created directory at : artifacts\prepare_callbacks\tensorboard_log_dir]
[2024-10-03 02:02:19,152 : INFO: common: Created directory at : artifacts\training]


Found 908 images belonging to 7 classes.
Found 2738 images belonging to 7 classes.
20/20 [==============================] - 191s 10s/step - loss: 28.2215 - accuracy: 0.1156 - val_loss: 17.7270 - val_accuracy: 0.2812


c:\ProgramDataanaconda3\Gesture\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
